In [ ]:
!pip install pyomo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from pyomo.environ import *
import sys
import os

# Install and configure CBC solver
if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

# Load data from Excel file
file_path = "/content/data for model_ba882.xlsx"
data = pd.read_excel(file_path)

Getting files...
Done
---------------------------------------------------
IDAES Extensions Build Versions
Solvers:  v3.4.2 20240811 ubuntu2204-x86_64
Library:  v3.4.2 20240811 ubuntu2204-x86_64


In [ ]:
# Extract relevant data
food_items = data['name'].tolist()
categories = data['Category'].tolist()
costs = data['Cost per Serving'].tolist()

# Nutritional values
calories = data['Calories (kcal)'].tolist()
protein = data['Protein (g)'].tolist()
fiber = data['Total Fiber (g)'].tolist()
vitamin_a = data['Vitamin A (IU)'].tolist()
vitamin_c = data['Vitamin C (mg)'].tolist()
calcium = data['Calcium (g)'].tolist()
iron = data['Iron (mg)'].tolist()

In [ ]:
# Function to solve the model for different trays
def solve_lunch_tray(food_items, categories, costs, calories, protein, fiber, vitamin_a, vitamin_c, calcium, iron, excluded_items):
    model = ConcreteModel()

    # Decision variables
    model.x = Var(food_items, domain=Binary)

    # Objective function: Minimize cost
    model.cost = Objective(expr=sum(model.x[item] * cost for item, cost in zip(food_items, costs)), sense=minimize)

    # Nutritional constraints
    model.constraints = ConstraintList()
    model.constraints.add(sum(model.x[item] * value for item, value in zip(food_items, calories)) >= 650)
    model.constraints.add(sum(model.x[item] * value for item, value in zip(food_items, protein)) >= 12)
    model.constraints.add(sum(model.x[item] * value for item, value in zip(food_items, fiber)) >= 5)
    model.constraints.add(sum(model.x[item] * value for item, value in zip(food_items, vitamin_a)) >= 350)
    model.constraints.add(sum(model.x[item] * value for item, value in zip(food_items, vitamin_c)) >= 20)
    model.constraints.add(sum(model.x[item] * value for item, value in zip(food_items, calcium)) >= 3.6)
    model.constraints.add(sum(model.x[item] * value for item, value in zip(food_items, iron)) >= 4)

    # Category constraints
    for category in set(categories):
        if category != "Beverages":
            model.constraints.add(
                sum(model.x[item] for item, cat in zip(food_items, categories) if cat == category) == 1
            )
        else:
            model.constraints.add(
                sum(model.x[item] for item, cat in zip(food_items, categories) if cat == category) >= 1
            )

    # Exclude items from previous trays except beverages
    for item in excluded_items:
        if categories[food_items.index(item)] != "Beverages":
            model.constraints.add(model.x[item] == 0)

    # Solve the model
    solver = SolverFactory("cbc")
    result = solver.solve(model)

    # Collect results
    selected_items = [item for item in food_items if model.x[item].value == 1]
    total_cost = model.cost()

    return selected_items, total_cost

In [ ]:
# Generate lunch trays and calculate costs for 75 students
total_cost_all_trays = 0
excluded_items = []
students = 75

for i in range(4):
    print(f"\nLunch Tray {i + 1}:")
    selected_items, total_cost = solve_lunch_tray(food_items, categories, costs, calories, protein, fiber, vitamin_a, vitamin_c, calcium, iron, excluded_items)
    tray_cost_for_students = total_cost * students
    total_cost_all_trays += tray_cost_for_students
    print("Selected items:", selected_items)
    print(f"Cost for one tray: ${total_cost:.2f}")
    print(f"Cost for {students} students: ${tray_cost_for_students:.2f}")


    excluded_items.extend([item for item in selected_items if categories[food_items.index(item)] != "Beverages"])


print(f"\nTotal cost for all trays for {students} students: ${total_cost_all_trays:.2f}")



Lunch Tray 1:
Selected items: ['milk', 'doughnut', 'tangerine', 'rice', 'egg', 'popcorn', 'carrot']
Cost for one tray: $0.98
Cost for 75 students: $73.50

Lunch Tray 2:
Selected items: ['milk', 'cookie', 'kiwifruit', 'spaghetti', 'bacon', 'bread', 'corn']
Cost for one tray: $1.30
Cost for 75 students: $97.50

Lunch Tray 3:
Selected items: ['milk', 'pancakes', 'banana', 'pizza', 'chicken', 'peanuts', 'hot pepper']
Cost for one tray: $2.24
Cost for 75 students: $168.00

Lunch Tray 4:
Selected items: ['milk', 'candy', 'grapes', 'hotdog', 'beef', 'croissant', 'potato']
Cost for one tray: $3.10
Cost for 75 students: $232.50

Total cost for all trays for 75 students: $571.50
